https://opendata.paris.fr/explore/dataset/paris-wi-fi-utilisation-des-hotspots-paris-wi-fi/information/?disjunctive.incomingzonelabel&disjunctive.incomingnetworklabel&disjunctive.device_portal_format&disjunctive.device_constructor_name&disjunctive.device_operating_system_name_version&disjunctive.device_browser_name_version&disjunctive.userlanguage

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import geopandas as gpd
from shapely.geometry import Point
import folium
import branca.colormap as cm
from branca.element import Template, MacroElement

In [2]:
data_df = pd.read_csv('paris-wi-fi-utilisation-des-hotspots-paris-wi-fi.csv', sep=';')
arrond_paris_gdf = gpd.read_file('arrondissements-paris.geojson', driver='GeoJSON')

In [3]:
data_df.head()

,Code Site,Date heure début,Date heure fin,Temps de session en secondes,Temps de sessions en minutes,Nom du site,Code postal,Type d'appareil,Constructeur appareil,Version système exploitation,Version navigateur,Donnée entrante (Octet),Donnée entrante (MégaOctet),Donnée sortante (Octet),Donnée sortante (MégaOctet),Paquets entrants,Paquets sortants,Langue utilisateur,geo_shape,geo_point_2d,Etat du site,Nom du site.1,Adresse,Nombre de bornes,Code postal.1
0,PW0134,2019-09-05T15:59:00+02:00,2019-09-05T17:59:00+02:00,7241,120.683333,BIB CHARLOTTE DELBO,75002,Ordinateur,Apple,Mac OS X 10.14.3,inconnu,844603314,805.476488,44462238,42.402494,583675,609233,Français,"{""type"": ""Point"", ""coordinates"": [2.3404344915...","48.8665075801,2.34043449149",Opérationnel,BIB CHARLOTTE DELBO,2 PASSAGE DES PETITS PERES,2.0,75002
1,PW0030,2019-09-05T16:07:00+02:00,2019-09-05T18:08:00+02:00,7231,120.516667,MVAC 12,75012,Ordinateur,autre,Windows 10,Firefox 69.0,48588507,46.337611,2677310,2.553282,35276,29076,Français,"{""type"": ""Point"", ""coordinates"": [2.3925067765...","48.8404878126,2.39250677653",Opérationnel,MVAC 12,181 AVENUE DAUMESNIL,6.0,75012
2,PW0181,2019-09-05T16:12:00+02:00,2019-09-05T16:15:00+02:00,183,3.050000,PONT NOTRE DAME-BERGES RD,75004,Mobile,autre,Android 7.1.1,Chrome 76.0,1800647,1.717231,190377,0.181558,1741,1533,Anglais,"{""type"": ""Point"", ""coordinates"": [2.3489230003...","48.8566875709,2.34892300033",Opérationnel,PONT NOTRE DAME-BERGES RD,BERGE DE SEINE RIVE DROITE - QUAI DE GESVRES 2,1.0,75004
3,PW0111,2019-09-05T16:14:00+02:00,2019-09-05T18:15:00+02:00,7260,121.000000,BIB BUFFON,75005,Mobile,autre,inconnu,Chrome 76.0,73516096,70.110413,8860475,8.450007,64607,61199,Anglais,"{""type"": ""Point"", ""coordinates"": [2.3618682436...","48.842546608,2.3618682436",Opérationnel,BIB BUFFON,15 B RUE BUFFON,3.0,75005
4,PW0161,2019-09-05T16:15:00+02:00,2019-09-05T16:18:00+02:00,149,2.483333,ESPLANADE DES INVALIDES,75007,Mobile,Samsung,Android 8.0.0,Chrome 76.0,928429,0.885419,604910,0.576887,1398,1385,Anglais,"{""type"": ""Point"", ""coordinates"": [2.312894264,...","48.8581897423,2.31289426397",Opérationnel,ESPLANADE DES INVALIDES,1 RUE PAUL ET JEAN LEROLLE,1.0,75007


In [4]:
neighbourhood_cp_dict = {
    'Batignolles-Monceau':75017,
    'Palais-Bourbon':75007,
    'Buttes-Chaumont':75019,
    'Opéra':75009,
    'Entrepôt':75010,
    'Gobelins':75013,
    'Vaugirard':75015,
    'Reuilly':75012,
    'Louvre':75001,
    'Luxembourg':75006,
    'Élysée':75008,
    'Temple':75003,
    'Ménilmontant':75020,
    'Panthéon':75005,
    'Passy':75016,
    'Observatoire':75014,
    'Popincourt':75011,
    'Bourse':75002,
    'Buttes-Montmartre':75018,
    'Hôtel-de-Ville':75004
}

arrond_paris_gdf['Code postal'] = arrond_paris_gdf['neighbourhood'].map(neighbourhood_cp_dict)

In [5]:
data_df = data_df[data_df['geo_shape'].isnull() == False]

data_df['latitude'], data_df['longitude'] = zip(*data_df['geo_point_2d'].apply(lambda x: x.split(',')))
data_df['latitude'] = data_df['latitude'].map(float)
data_df['longitude'] = data_df['longitude'].map(float)

In [6]:
data_df.loc[(data_df['Code Site']=='PW0136') &
            (data_df['Adresse']=='18 BOULEVARD SERURIER'), 'Code postal'] = 75019
data_df.loc[(data_df['Code Site']=='PW0136') &
            (data_df['Adresse']=='6-12 RUE DES PRETRES ST-SEVERIN'), 'Code postal'] = 75005
data_df.loc[(data_df['Code Site']=='PW0144') &
            (data_df['Adresse']=='7 AVENUE VELASQUEZ'), 'Code postal'] = 75008
data_df.loc[(data_df['Code Site']=='PW0144') &
            (data_df['Adresse']=='1 RUE REYNALDO HAHN'), 'Code postal'] = 75020

data_df.loc[(data_df['Code Site']=='PW0001'), 'Code postal'] = 75019
data_df.loc[(data_df['Code Site']=='PW0002'), 'Code postal'] = 75019
data_df.loc[(data_df['Code Site']=='PW0006'), 'Code postal'] = 75018
data_df.loc[(data_df['Code Site']=='PW0009'), 'Code postal'] = 75018
data_df.loc[(data_df['Code Site']=='PW0010'), 'Code postal'] = 75010
data_df.loc[(data_df['Code Site']=='PW0011'), 'Code postal'] = 75011
data_df.loc[(data_df['Code Site']=='PW0014'), 'Code postal'] = 75015
data_df.loc[(data_df['Code Site']=='PW0019'), 'Code postal'] = 75003
data_df.loc[(data_df['Code Site']=='PW0020'), 'Code postal'] = 75001
data_df.loc[(data_df['Code Site']=='PW0021'), 'Code postal'] = 75005
data_df.loc[(data_df['Code Site']=='PW0022-1'), 'Code postal'] = 75007
data_df.loc[(data_df['Code Site']=='PW0023'), 'Code postal'] = 75018
data_df.loc[(data_df['Code Site']=='PW0027'), 'Code postal'] = 75007
data_df.loc[(data_df['Code Site']=='PW0028'), 'Code postal'] = 75010
data_df.loc[(data_df['Code Site']=='PW0030'), 'Code postal'] = 75012
data_df.loc[(data_df['Code Site']=='PW0034'), 'Code postal'] = 75002
data_df.loc[(data_df['Code Site']=='PW0036'), 'Code postal'] = 75002
data_df.loc[(data_df['Code Site']=='PW0043'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0044'), 'Code postal'] = 75003
data_df.loc[(data_df['Code Site']=='PW0045'), 'Code postal'] = 75020
data_df.loc[(data_df['Code Site']=='PW0046'), 'Code postal'] = 75015
data_df.loc[(data_df['Code Site']=='PW0048'), 'Code postal'] = 75020
data_df.loc[(data_df['Code Site']=='PW0051'), 'Code postal'] = 75012
data_df.loc[(data_df['Code Site']=='PW0052'), 'Code postal'] = 75011
data_df.loc[(data_df['Code Site']=='PW0054'), 'Code postal'] = 75014
data_df.loc[(data_df['Code Site']=='PW0055'), 'Code postal'] = 75020
data_df.loc[(data_df['Code Site']=='PW0056'), 'Code postal'] = 75011
data_df.loc[(data_df['Code Site']=='PW0057'), 'Code postal'] = 75020
data_df.loc[(data_df['Code Site']=='PW0058'), 'Code postal'] = 75012
data_df.loc[(data_df['Code Site']=='PW0058-1'), 'Code postal'] = 75012
data_df.loc[(data_df['Code Site']=='PW0058-3'), 'Code postal'] = 75012
data_df.loc[(data_df['Code Site']=='PW0066'), 'Code postal'] = 75013
data_df.loc[(data_df['Code Site']=='PW0069'), 'Code postal'] = 75013
data_df.loc[(data_df['Code Site']=='PW0070'), 'Code postal'] = 75003
data_df.loc[(data_df['Code Site']=='PW0072'), 'Code postal'] = 75020
data_df.loc[(data_df['Code Site']=='PW0089'), 'Code postal'] = 75018
data_df.loc[(data_df['Code Site']=='PW0104'), 'Code postal'] = 75017
data_df.loc[(data_df['Code Site']=='PW0108'), 'Code postal'] = 75016
data_df.loc[(data_df['Code Site']=='PW0110'), 'Code postal'] = 75001
data_df.loc[(data_df['Code Site']=='PW0111'), 'Code postal'] = 75005
data_df.loc[(data_df['Code Site']=='PW0112'), 'Code postal'] = 75001
data_df.loc[(data_df['Code Site']=='PW0115'), 'Code postal'] = 75005
data_df.loc[(data_df['Code Site']=='PW0116'), 'Code postal'] = 75010
data_df.loc[(data_df['Code Site']=='PW0117'), 'Code postal'] = 75015
data_df.loc[(data_df['Code Site']=='PW0118'), 'Code postal'] = 75006
data_df.loc[(data_df['Code Site']=='PW0119'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0121'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0122-1'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0122-3'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0122-4'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0122-5'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0123'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0123-1'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0124'), 'Code postal'] = 75008
data_df.loc[(data_df['Code Site']=='PW0132'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0132-2'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0134'), 'Code postal'] = 75002
data_df.loc[(data_df['Code Site']=='PW0136'), 'Code postal'] = 75019
data_df.loc[(data_df['Code Site']=='PW0137'), 'Code postal'] = 75015
data_df.loc[(data_df['Code Site']=='PW0138'), 'Code postal'] = 75005
data_df.loc[(data_df['Code Site']=='PW0139'), 'Code postal'] = 75019
data_df.loc[(data_df['Code Site']=='PW0141'), 'Code postal'] = 75012
data_df.loc[(data_df['Code Site']=='PW0142'), 'Code postal'] = 75009
data_df.loc[(data_df['Code Site']=='PW0143'), 'Code postal'] = 75001
data_df.loc[(data_df['Code Site']=='PW0144'), 'Code postal'] = 75020
data_df.loc[(data_df['Code Site']=='PW0146'), 'Code postal'] = 75006
data_df.loc[(data_df['Code Site']=='PW0147'), 'Code postal'] = 75010
data_df.loc[(data_df['Code Site']=='PW0149'), 'Code postal'] = 75001
data_df.loc[(data_df['Code Site']=='PW0150'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0151-1'), 'Code postal'] = 75008
data_df.loc[(data_df['Code Site']=='PW0152'), 'Code postal'] = 75011
data_df.loc[(data_df['Code Site']=='PW0153'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0154'), 'Code postal'] = 75010
data_df.loc[(data_df['Code Site']=='PW0155'), 'Code postal'] = 75005
data_df.loc[(data_df['Code Site']=='PW0156'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0158'), 'Code postal'] = 75001
data_df.loc[(data_df['Code Site']=='PW0159'), 'Code postal'] = 75019
data_df.loc[(data_df['Code Site']=='PW0161'), 'Code postal'] = 75007
data_df.loc[(data_df['Code Site']=='PW0163'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0164'), 'Code postal'] = 75010
data_df.loc[(data_df['Code Site']=='PW0165'), 'Code postal'] = 75007
data_df.loc[(data_df['Code Site']=='PW0165-1'), 'Code postal'] = 75007
data_df.loc[(data_df['Code Site']=='PW0166'), 'Code postal'] = 75017
data_df.loc[(data_df['Code Site']=='PW0167'), 'Code postal'] = 75019
data_df.loc[(data_df['Code Site']=='PW0169-1'), 'Code postal'] = 75013
data_df.loc[(data_df['Code Site']=='PW0170'), 'Code postal'] = 75018
data_df.loc[(data_df['Code Site']=='PW0170-1'), 'Code postal'] = 75018
data_df.loc[(data_df['Code Site']=='PW0172'), 'Code postal'] = 75013
data_df.loc[(data_df['Code Site']=='PW0175'), 'Code postal'] = 75016
data_df.loc[(data_df['Code Site']=='PW0176'), 'Code postal'] = 75009
data_df.loc[(data_df['Code Site']=='PW0177'), 'Code postal'] = 75011
data_df.loc[(data_df['Code Site']=='PW0180'), 'Code postal'] = 75013
data_df.loc[(data_df['Code Site']=='PW0181'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0182'), 'Code postal'] = 75019
data_df.loc[(data_df['Code Site']=='PW0187'), 'Code postal'] = 75020
data_df.loc[(data_df['Code Site']=='PW0189'), 'Code postal'] = 75007
data_df.loc[(data_df['Code Site']=='PW0190'), 'Code postal'] = 75007
data_df.loc[(data_df['Code Site']=='PW0191'), 'Code postal'] = 75004
data_df.loc[(data_df['Code Site']=='PW0193'), 'Code postal'] = 75001
data_df.loc[(data_df['Code Site']=='PW0194'), 'Code postal'] = 75010
data_df.loc[(data_df['Code Site']=='PW0196'), 'Code postal'] = 75008
data_df.loc[(data_df['Code Site']=='PW0198'), 'Code postal'] = 75018
data_df.loc[(data_df['Code Site']=='PW0199'), 'Code postal'] = 75014
data_df.loc[(data_df['Code Site']=='PW0201'), 'Code postal'] = 75014
data_df.loc[(data_df['Code Site']=='PW0205'), 'Code postal'] = 75011
data_df.loc[(data_df['Code Site']=='PW0206'), 'Code postal'] = 75011
data_df.loc[(data_df['Code Site']=='PW0207'), 'Code postal'] = 75007
data_df.loc[(data_df['Code Site']=='PW0208'), 'Code postal'] = 75006
data_df.loc[(data_df['Code Site']=='PW0210'), 'Code postal'] = 75019
data_df.loc[(data_df['Code Site']=='PW0211'), 'Code postal'] = 75016
data_df.loc[(data_df['Code Site']=='PW0213'), 'Code postal'] = 75014
data_df.loc[(data_df['Code Site']=='PW0216'), 'Code postal'] = 75012

In [7]:
data_df.head(2)

,Code Site,Date heure début,Date heure fin,Temps de session en secondes,Temps de sessions en minutes,Nom du site,Code postal,Type d'appareil,Constructeur appareil,Version système exploitation,Version navigateur,Donnée entrante (Octet),Donnée entrante (MégaOctet),Donnée sortante (Octet),Donnée sortante (MégaOctet),Paquets entrants,Paquets sortants,Langue utilisateur,geo_shape,geo_point_2d,Etat du site,Nom du site.1,Adresse,Nombre de bornes,Code postal.1,latitude,longitude
0,PW0134,2019-09-05T15:59:00+02:00,2019-09-05T17:59:00+02:00,7241,120.683333,BIB CHARLOTTE DELBO,75002,Ordinateur,Apple,Mac OS X 10.14.3,inconnu,844603314,805.476488,44462238,42.402494,583675,609233,Français,"{""type"": ""Point"", ""coordinates"": [2.3404344915...","48.8665075801,2.34043449149",Opérationnel,BIB CHARLOTTE DELBO,2 PASSAGE DES PETITS PERES,2.0,75002,48.866508,2.340434
1,PW0030,2019-09-05T16:07:00+02:00,2019-09-05T18:08:00+02:00,7231,120.516667,MVAC 12,75012,Ordinateur,autre,Windows 10,Firefox 69.0,48588507,46.337611,2677310,2.553282,35276,29076,Français,"{""type"": ""Point"", ""coordinates"": [2.3925067765...","48.8404878126,2.39250677653",Opérationnel,MVAC 12,181 AVENUE DAUMESNIL,6.0,75012,48.840488,2.392507


In [8]:
district_lang_df = data_df.groupby(['Code postal',
                                    'Langue utilisateur'])['Langue utilisateur'].count().to_frame()
district_lang_df = district_lang_df.rename(columns={'Langue utilisateur':'nb_connetion'})
district_lang_df = district_lang_df.sort_values('nb_connetion', ascending=False)
district_lang_df = district_lang_df.reset_index()

district_lang_df = pd.merge(district_lang_df,
                            arrond_paris_gdf[['Code postal', 'geometry']],
                            on='Code postal')

In [9]:
site_lang_df = data_df.groupby(['Code Site', 'Langue utilisateur',
                                'Adresse', 'latitude',
                                'longitude'])['Code Site'].count().to_frame()
site_lang_df = site_lang_df.rename(columns={'Code Site':'nb_connetion'})
site_lang_df = site_lang_df.sort_values('nb_connetion', ascending=False)
site_lang_df = site_lang_df.reset_index()


In [10]:
district_device_df = data_df.groupby(['Code postal',
                                      'Constructeur appareil'])['Constructeur appareil'].count().to_frame()
district_device_df = district_device_df.rename(columns={'Constructeur appareil':'nb_connetion'})
district_device_df = district_device_df.sort_values('nb_connetion', ascending=False)
district_device_df = district_device_df.reset_index()

district_device_df = pd.merge(district_device_df,
                              arrond_paris_gdf[['Code postal', 'geometry']],
                              on='Code postal')

In [11]:
site_device_df = data_df.groupby(['Code Site', 'Constructeur appareil',
                                  'Adresse', 'latitude',
                                  'longitude'])['Code Site'].count().to_frame()
site_device_df = site_device_df.rename(columns={'Code Site':'nb_connetion'})
site_device_df = site_device_df.sort_values('nb_connetion', ascending=False)
site_device_df = site_device_df.reset_index()


## Visualisation

### Per site : language used

In [12]:
district_cn_df = district_lang_df[district_lang_df['Langue utilisateur']=='Chinois simplifié']
district_ru_df = district_lang_df[district_lang_df['Langue utilisateur']=='Russe']
district_ar_df = district_lang_df[district_lang_df['Langue utilisateur']=='Arabe']
district_jp_df = district_lang_df[district_lang_df['Langue utilisateur']=='Japonais']
district_cr_df = district_lang_df[district_lang_df['Langue utilisateur']=='Coréen']

In [13]:
crs = {'init': 'epsg:4326'}
arrond_paris_gdf = gpd.GeoDataFrame(arrond_paris_gdf, crs=crs)
district_cn_df = gpd.GeoDataFrame(district_cn_df, crs=crs)
district_ru_df = gpd.GeoDataFrame(district_ru_df, crs=crs)
district_ar_df = gpd.GeoDataFrame(district_ar_df, crs=crs)
district_jp_df = gpd.GeoDataFrame(district_jp_df, crs=crs)
district_cr_df = gpd.GeoDataFrame(district_cr_df, crs=crs)

site_lang_geometry = [Point(xy) for xy in zip(site_lang_df.longitude, site_lang_df.latitude)]
site_lang_gdf = gpd.GeoDataFrame(site_lang_df, crs=crs, geometry=site_lang_geometry)

In [14]:
site_cn_map = folium.Map(location=[48.856614, 2.3522219], zoom_start = 12, tiles='cartodbpositron')

folium.GeoJson(
    arrond_paris_gdf,
    style_function = lambda x: {
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0
    },
    name='Paris').add_to(site_cn_map)

for i, v in site_lang_gdf[site_lang_gdf['Langue utilisateur']=='Chinois simplifié'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Language : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Langue utilisateur'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 10 if v['nb_connetion'] <= 100 else 11,
                            tooltip=popup,
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(site_cn_map)


# folium.LayerControl(collapsed=False).add_to(site_cn_map)

In [15]:
site_cn_map

In [16]:
# colormap_cn = cm.StepColormap(
#     colors=['#F6DF7B', '#F6C74D',
#             '#F68A53', '#AF3D3D', '#842F41'],
#     vmin=min(district_ru_df['nb_connetion']),
#     vmax=max(district_ru_df['nb_connetion']),
#     index=[33, 128, 175, 257, 470, 946])

colormap_cn = cm.StepColormap(
    colors=['#FFB41C', '#FF8B01',
            '#D32122', '#800021', '#63001A'],
    vmin=min(district_cn_df['nb_connetion']),
    vmax=max(district_cn_df['nb_connetion']),
    index=[33, 128, 175, 257, 470, 946])


template_cn = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body> 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Connection count</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FFB41C;opacity:1;'></span>1 - 128</li>
    <li><span style='background:#FF8B01;opacity:1;'></span>129 - 175</li>
    <li><span style='background:#D32122;opacity:1;'></span>176 - 257</li>
    <li><span style='background:#800021;opacity:1;'></span>258 - 470</li>
    <li><span style='background:#63001A;opacity:1;'></span>> 470</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

template_cn_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 60px; top: 0px;'>
<div class='legend-title'>Uses of WIFI hotspots in Paris by user language: Chinese</div>
</div>

</body>
</html>

{% endmacro %}"""

In [17]:
district_cn_map = folium.Map(location=[48.856614, 2.3522219],
                           zoom_start=12, tiles='cartodbpositron')

folium.GeoJson(
    district_cn_df,
    style_function=lambda x: {
    'fillColor': colormap_cn(x['properties']['nb_connetion']),
    'color': 'black',
    'weight': 2,
    'fillOpacity': 0.8
},
    tooltip=folium.GeoJsonTooltip(
        fields=['Code postal', 'nb_connetion'],
        aliases=['Code postal', 'Nb connection'],
        localize=False
    ),
    name='Chinese').add_to(district_cn_map)

for i, v in site_lang_gdf[site_lang_gdf['Langue utilisateur']=='Chinois simplifié'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Language : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Langue utilisateur'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 10 if v['nb_connetion'] <= 100 else 11,
                            tooltip=popup,
                            color='#FF7B7B',# 'orange', FF7B7B E54B4B
                            fill_color='#FF7B7B',
                            fill=True).add_to(district_cn_map)


folium.map.CustomPane('labels').add_to(district_cn_map)
folium.TileLayer('CartoDBPositronOnlyLabels',
                 pane='labels').add_to(district_cn_map)

macro = MacroElement()
macro._template = Template(template_cn)
macro2 = MacroElement()
macro2._template = Template(template_cn_title)
district_cn_map.get_root().add_child(macro)
district_cn_map.get_root().add_child(macro2)

In [18]:
colormap_ru = cm.StepColormap(
    colors=['#D0F1F7', '#A1E3F7', '#90D1F3', '#6DACEA', '#3C65D1'],
    vmin=min(district_ru_df['nb_connetion']),
    vmax=max(district_ru_df['nb_connetion']),
    index=[30, 68, 284, 428, 846, 3201])


template_ru = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body> 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Connection count</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#D0F1F7;opacity:1;'></span>1 - 68</li>
    <li><span style='background:#A1E3F7;opacity:1;'></span>69 - 284</li>
    <li><span style='background:#90D1F3;opacity:1;'></span>285 - 428</li>
    <li><span style='background:#6DACEA;opacity:1;'></span>429 - 846</li>
    <li><span style='background:#3C65D1;opacity:1;'></span>> 846</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

template_ru_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 60px; top: 0px;'>
<div class='legend-title'>Uses of WIFI hotspots in Paris by user language: Russian</div>
</div>

</body>
</html>

{% endmacro %}"""

In [19]:
district_ru_map = folium.Map(location=[48.856614, 2.3522219],
                           zoom_start=12, tiles='cartodbpositron') # 'cartodbpositron' Stamen Toner

folium.GeoJson(
    district_ru_df,
    style_function=lambda x: {
    'fillColor': colormap_ru(x['properties']['nb_connetion']),
    'color': 'black',
    'weight': 2,
    'fillOpacity': 1
},
    tooltip=folium.GeoJsonTooltip(
        fields=['Code postal', 'nb_connetion'],
        aliases=['Code postal', 'Nb connection'],
        localize=False
    ),
    name='Russe').add_to(district_ru_map)

for i, v in site_lang_gdf[site_lang_gdf['Langue utilisateur']=='Russe'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Language : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Langue utilisateur'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 10 if v['nb_connetion'] <= 100 else 11,
                            tooltip=popup,
                            color='#87A1E8',# 87A1E8 F8FEFF
                            fill_color='#87A1E8',
                            fill=True).add_to(district_ru_map)

folium.map.CustomPane('labels').add_to(district_ru_map)
folium.TileLayer('CartoDBPositronOnlyLabels',
                 pane='labels').add_to(district_ru_map)

macro = MacroElement()
macro._template = Template(template_ru)
macro2 = MacroElement()
macro2._template = Template(template_ru_title)
district_ru_map.get_root().add_child(macro)
district_ru_map.get_root().add_child(macro2)

In [20]:
colormap_ar = cm.StepColormap(
    colors=['#D3E502', '#9BC004', '#5D9A05','#277903', '#036406'],
    vmin=min(district_ar_df['nb_connetion']),
    vmax=max(district_ar_df['nb_connetion']),
    index=[3, 15, 33, 95, 179, 2373])


template_ar = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body> 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Connection count</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#D3E502;opacity:1;'></span>1 - 15</li>
    <li><span style='background:#9BC004;opacity:1;'></span>16 - 33</li>
    <li><span style='background:#5D9A05;opacity:1;'></span>34 - 95</li>
    <li><span style='background:#277903;opacity:1;'></span>96 - 179</li>
    <li><span style='background:#036406;opacity:1;'></span>> 179</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

template_ar_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 60px; top: 0px;'>
<div class='legend-title'>Uses of WIFI hotspots in Paris by user language: Arab</div>
</div>

</body>
</html>

{% endmacro %}"""

In [21]:
district_ar_map = folium.Map(location=[48.856614, 2.3522219],
                           zoom_start=12, tiles='cartodbpositron')

folium.GeoJson(
    district_ar_df,
    style_function=lambda x: {
    'fillColor': colormap_ar(x['properties']['nb_connetion']),
    'color': 'black',
    'weight': 2,
    'fillOpacity': 0.8
},
    tooltip=folium.GeoJsonTooltip(
        fields=['Code postal', 'nb_connetion'],
        aliases=['Code postal', 'Nb connection'],
        localize=False
    ),
    name='Arab').add_to(district_ar_map)

for i, v in site_lang_gdf[site_lang_gdf['Langue utilisateur']=='Arabe'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Language : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Langue utilisateur'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 10 if v['nb_connetion'] <= 100 else 11,
                            tooltip=popup,
                            color='#FFFE00', # FFFE00
                            fill_color='#FFFE00',
                            fill=True).add_to(district_ar_map)

folium.map.CustomPane('labels').add_to(district_ar_map)
folium.TileLayer('CartoDBPositronOnlyLabels',
                 pane='labels').add_to(district_ar_map)

macro = MacroElement()
macro._template = Template(template_ar)
macro2 = MacroElement()
macro2._template = Template(template_ar_title)
district_ar_map.get_root().add_child(macro)
district_ar_map.get_root().add_child(macro2)

In [22]:
colormap_jp = cm.StepColormap(
    colors=['#fef4f4', '#fdeff2',
            '#e9dfe5', '#e4d2d8', '#f6bfbc'],
    vmin=min(district_jp_df['nb_connetion']),
    vmax=max(district_jp_df['nb_connetion']),
    index=[9, 47, 67, 73, 207, 597])


template_jp = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body> 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Connection count</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#fef4f4;opacity:1;'></span>1 - 47</li>
    <li><span style='background:#fdeff2;opacity:1;'></span>48 - 67</li>
    <li><span style='background:#e9dfe5;opacity:1;'></span>68 - 73</li>
    <li><span style='background:#e4d2d8;opacity:1;'></span>74 - 207</li>
    <li><span style='background:#f6bfbc;opacity:1;'></span>> 207</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

template_jp_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 60px; top: 0px;'>
<div class='legend-title'>Uses of WIFI hotspots in Paris by user language: Japanese</div>
</div>

</body>
</html>

{% endmacro %}"""

In [23]:
district_jp_map = folium.Map(location=[48.856614, 2.3522219],
                           zoom_start=12, tiles='cartodbpositron')

folium.GeoJson(
    district_jp_df,
    style_function=lambda x: {
    'fillColor': colormap_jp(x['properties']['nb_connetion']),
    'color': 'black',
    'weight': 2,
    'fillOpacity': 1
},
    tooltip=folium.GeoJsonTooltip(
        fields=['Code postal', 'nb_connetion'],
        aliases=['Code postal', 'Nb connection'],
        localize=False
    ),
    name='Japon').add_to(district_jp_map)

for i, v in site_lang_gdf[site_lang_gdf['Langue utilisateur']=='Japonais'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Language : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Langue utilisateur'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 10 if v['nb_connetion'] <= 100 else 11,
                            tooltip=popup,
                            color='#D18884', # FFFE00 D18884
                            fill_color='#D18884',
                            fill=True).add_to(district_jp_map)

folium.map.CustomPane('labels').add_to(district_jp_map)
folium.TileLayer('CartoDBPositronOnlyLabels',
                 pane='labels').add_to(district_jp_map)

macro = MacroElement()
macro._template = Template(template_jp)
macro2 = MacroElement()
macro2._template = Template(template_jp_title)
district_jp_map.get_root().add_child(macro)
district_jp_map.get_root().add_child(macro2)


In [24]:
colormap_cr = cm.StepColormap(
    colors=['#FFFDD0', '#FFF38D',
            '#FBD160', '#E5AA22', '#C68E07'],
    vmin=min(district_cr_df['nb_connetion']),
    vmax=max(district_cr_df['nb_connetion']),
    index=[7, 21, 27, 58, 160, 637])



template_cr = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body> 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Connection count</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FFFDD0;opacity:1;'></span>1 - 21</li>
    <li><span style='background:#FFF38D;opacity:1;'></span>22 - 27</li>
    <li><span style='background:#FBD160;opacity:1;'></span>38 - 58</li>
    <li><span style='background:#E5AA22;opacity:1;'></span>59 - 160</li>
    <li><span style='background:#C68E07;opacity:1;'></span>> 160</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

template_cr_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 60px; top: 0px;'>
<div class='legend-title'>Uses of WIFI hotspots in Paris by user language: Korean</div>
</div>

</body>
</html>

{% endmacro %}"""

In [25]:
district_cr_map = folium.Map(location=[48.856614, 2.3522219],
                           zoom_start=12, tiles='cartodbpositron')

folium.GeoJson(
    district_cr_df,
    style_function=lambda x: {
    'fillColor': colormap_cr(x['properties']['nb_connetion']),
    'color': 'black',
    'weight': 2,
    'fillOpacity': 0.7
},
    tooltip=folium.GeoJsonTooltip(
        fields=['Code postal', 'nb_connetion'],
        aliases=['Code postal', 'Nb connection'],
        localize=False
    ),
    name='Korea').add_to(district_cr_map)

for i, v in site_lang_gdf[site_lang_gdf['Langue utilisateur']=='Coréen'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Language : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Langue utilisateur'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 10 if v['nb_connetion'] <= 100 else 11,
                            tooltip=popup,
                            color='#FFFD00', # FAE452 FFD679 FFFD00
                            fill_color='#FFFD00',
                            fill=True).add_to(district_cr_map)

folium.map.CustomPane('labels').add_to(district_cr_map)
folium.TileLayer('CartoDBPositronOnlyLabels',
                 pane='labels').add_to(district_cr_map)

macro = MacroElement()
macro._template = Template(template_cr)
macro2 = MacroElement()
macro2._template = Template(template_cr_title)
district_cr_map.get_root().add_child(macro)
district_cr_map.get_root().add_child(macro2)

### Per site : device used

In [26]:
district_apple_df = district_device_df[district_device_df['Constructeur appareil']=='Apple']
district_samsung_df = district_device_df[district_device_df['Constructeur appareil']=='Samsung']
district_huawei_df = district_device_df[district_device_df['Constructeur appareil']=='HUAWEI']
district_nokia_df = district_device_df[district_device_df['Constructeur appareil']=='Nokia']




In [27]:
crs = {'init': 'epsg:4326'}
district_apple_df = gpd.GeoDataFrame(district_apple_df, crs=crs)
district_samsung_df = gpd.GeoDataFrame(district_samsung_df, crs=crs)
district_huawei_df = gpd.GeoDataFrame(district_huawei_df, crs=crs)
district_nokia_df = gpd.GeoDataFrame(district_nokia_df, crs=crs)

site_device_geometry = [Point(xy) for xy in zip(site_device_df.longitude, site_device_df.latitude)]
site_device_gdf = gpd.GeoDataFrame(site_device_df, crs=crs, geometry=site_device_geometry)

In [28]:
district_device_df.head()

,Code postal,Constructeur appareil,nb_connetion,geometry
0,75004,Apple,39217,"(POLYGON ((2.368512 48.855734, 2.369003 48.853..."
1,75004,autre,29858,"(POLYGON ((2.368512 48.855734, 2.369003 48.853..."
2,75004,Samsung,6523,"(POLYGON ((2.368512 48.855734, 2.369003 48.853..."
3,75004,HUAWEI,682,"(POLYGON ((2.368512 48.855734, 2.369003 48.853..."
4,75004,Asus,247,"(POLYGON ((2.368512 48.855734, 2.369003 48.853..."


In [29]:
site_device_df.head(2)

,Code Site,Constructeur appareil,Adresse,latitude,longitude,nb_connetion,geometry
0,PW0147,Apple,8 RUE LEON SCHWARTZENBERG,48.87543,2.353667,12951,POINT (2.35366677297 48.8754298708)
1,PW0147,autre,8 RUE LEON SCHWARTZENBERG,48.87543,2.353667,11148,POINT (2.35366677297 48.8754298708)


In [30]:
district_device_df.groupby(['Constructeur appareil'])['nb_connetion'].sum().sort_values()

Constructeur appareil
ARCHOS          6
Sony           60
RIM           210
HTC           832
LG           2059
Nokia        3167
Asus         3848
HUAWEI       6563
Samsung     48226
Apple      270114
autre      276997
Name: nb_connetion, dtype: int64

In [31]:
colormap_apple = cm.StepColormap(
    colors=['#ede9a3', '#d7c37a',
            '#55a44e', '#1b8057', '#003e21'],
    vmin=min(district_apple_df['nb_connetion']),
    vmax=max(district_apple_df['nb_connetion']),
    index=[2417, 6436, 10640, 13674, 18759, 39217])

# colormap_apple = cm.StepColormap(
#     colors=['#FFD853', '#FFBA47',
#             '#FF9C48', '#FF5E3C', '#CF0A2C'],
#     vmin=min(district_apple_df['nb_connetion']),
#     vmax=max(district_apple_df['nb_connetion']),
#     index=[2417, 6436, 10640, 13674, 18759, 39217])



template_apple = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body> 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Connection count</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#ede9a3;opacity:1;'></span>1 - 6436</li>
    <li><span style='background:#d7c37a;opacity:1;'></span>6437 - 10640</li>
    <li><span style='background:#55a44e;opacity:1;'></span>10641 - 13674</li>
    <li><span style='background:#1b8057;opacity:1;'></span>13675 - 18759</li>
    <li><span style='background:#003e21;opacity:1;'></span>> 18759</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

template_apple_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 60px; top: 0px;'>
<div class='legend-title'>Uses of WIFI hotspots in Paris by device: Apple</div>
</div>

</body>
</html>

{% endmacro %}"""

In [32]:
district_apple_map = folium.Map(location=[48.856614, 2.3522219],
                           zoom_start=12, tiles='cartodbpositron')

folium.GeoJson(
    district_apple_df,
    style_function=lambda x: {
    'fillColor': colormap_apple(x['properties']['nb_connetion']),
    'color': 'black',
    'weight': 2,
    'fillOpacity': 0.8
},
    tooltip=folium.GeoJsonTooltip(
        fields=['Code postal', 'nb_connetion'],
        aliases=['Code postal', 'Nb connection'],
        localize=False
    ),
    name='Apple').add_to(district_apple_map)

for i, v in site_device_df[site_device_df['Constructeur appareil']=='Apple'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Device : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Constructeur appareil'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 1000,
                            tooltip=popup,
                            color='#D0E9CD', # BFE3C1 D0E9CD
                            fill_color='#D0E9CD',
                            fill=True).add_to(district_apple_map)

folium.map.CustomPane('labels').add_to(district_apple_map)
folium.TileLayer('CartoDBPositronOnlyLabels',
                 pane='labels').add_to(district_apple_map)

macro = MacroElement()
macro._template = Template(template_apple)
macro2 = MacroElement()
macro2._template = Template(template_apple_title)
district_apple_map.get_root().add_child(macro)
district_apple_map.get_root().add_child(macro2)

In [33]:
colormap_samsung = cm.StepColormap(
    colors=['#FFFFB7', '#FFF192',
            '#FFEA61', '#FFDD3C', '#FFD400'],
    vmin=min(district_samsung_df['nb_connetion']),
    vmax=max(district_samsung_df['nb_connetion']),
    index=[419, 800, 1730, 2645, 3981, 6523])


template_samsung = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body> 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Connection count</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FFFFB7;opacity:1;'></span>1 - 800</li>
    <li><span style='background:#FFF192;opacity:1;'></span>801 - 1730</li>
    <li><span style='background:#FFEA61;opacity:1;'></span>1731 - 2645</li>
    <li><span style='background:#FFDD3C;opacity:1;'></span>2646 - 3981</li>
    <li><span style='background:#FFD400;opacity:1;'></span>> 3981</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

template_samsung_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 60px; top: 0px;'>
<div class='legend-title'>Uses of WIFI hotspots in Paris by device: Samsung</div>
</div>

</body>
</html>

{% endmacro %}"""

In [34]:
district_samsung_map = folium.Map(location=[48.856614, 2.3522219],
                           zoom_start=12, tiles='cartodbpositron')

folium.GeoJson(
    district_samsung_df,
    style_function=lambda x: {
    'fillColor': colormap_samsung(x['properties']['nb_connetion']),
    'color': 'black',
    'weight': 2,
    'fillOpacity': 1
},
    tooltip=folium.GeoJsonTooltip(
        fields=['Code postal', 'nb_connetion'],
        aliases=['Code postal', 'Nb connection'],
        localize=False
    ),
    name='Samsung').add_to(district_samsung_map)

for i, v in site_device_df[site_device_df['Constructeur appareil']=='Samsung'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Device : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Constructeur appareil'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 100,
                            tooltip=popup,
                            color='#f08b33', # B69704 FFFB42 yellow 9199be
                            fill_color='#f08b33',
                            fill=True).add_to(district_samsung_map)

folium.map.CustomPane('labels').add_to(district_samsung_map)
folium.TileLayer('CartoDBPositronOnlyLabels',
                 pane='labels').add_to(district_samsung_map)

macro = MacroElement()
macro._template = Template(template_samsung)
macro2 = MacroElement()
macro2._template = Template(template_samsung_title)
district_samsung_map.get_root().add_child(macro)
district_samsung_map.get_root().add_child(macro2)

In [35]:
colormap_huawei = cm.StepColormap(
    colors=['#FFD853', '#FFBA47',
            '#FF9840', '#FF713C', '#CF0A2C'],
    vmin=min(district_huawei_df['nb_connetion']),
    vmax=max(district_huawei_df['nb_connetion']),
    index=[12, 122, 237, 411, 499, 698])

# FF5E3C
template_huawei = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body> 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Connection count</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FFD853;opacity:1;'></span>1 - 122</li>
    <li><span style='background:#FFBA47;opacity:1;'></span>123 - 237</li>
    <li><span style='background:#FF9840;opacity:1;'></span>238 - 411</li>
    <li><span style='background:#FF713C;opacity:1;'></span>412 - 499</li>
    <li><span style='background:#CF0A2C;opacity:1;'></span>> 499</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

template_huawei_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 60px; top: 0px;'>
<div class='legend-title'>Uses of WIFI hotspots in Paris by device: Huawei</div>
</div>

</body>
</html>

{% endmacro %}"""

In [36]:
district_huawei_map = folium.Map(location=[48.856614, 2.3522219],
                           zoom_start=12, tiles='cartodbpositron')

folium.GeoJson(
    district_huawei_df,
    style_function=lambda x: {
    'fillColor': colormap_huawei(x['properties']['nb_connetion']),
    'color': '#232527',
    'weight': 2,
    'fillOpacity': 0.8
},
    tooltip=folium.GeoJsonTooltip(
        fields=['Code postal', 'nb_connetion'],
        aliases=['Code postal', 'Nb connection'],
        localize=False
    ),
    name='huawei').add_to(district_huawei_map)

for i, v in site_device_df[site_device_df['Constructeur appareil']=='HUAWEI'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Device : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Constructeur appareil'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 20,
                            tooltip=popup,
                            color='#FFFE64',
                            fill_color='#FFFE64',
                            fill=True).add_to(district_huawei_map)

folium.map.CustomPane('labels').add_to(district_huawei_map)
folium.TileLayer('CartoDBPositronOnlyLabels',
                 pane='labels').add_to(district_huawei_map)

macro = MacroElement()
macro._template = Template(template_huawei)
macro2 = MacroElement()
macro2._template = Template(template_huawei_title)
district_huawei_map.get_root().add_child(macro)
district_huawei_map.get_root().add_child(macro2)

In [37]:
colormap_nokia = cm.StepColormap(
    colors=['#9DC8E6', '#7FBDD9',
            '#62A7C5', '#348CB1', '#007AA8'],
    vmin=min(district_nokia_df['nb_connetion']),
    vmax=max(district_nokia_df['nb_connetion']),
    index=[4, 53, 114, 132, 285, 396])


template_nokia = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body> 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Connection count</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#9DC8E6;opacity:1;'></span>1 - 53</li>
    <li><span style='background:#7FBDD9;opacity:1;'></span>54 - 114</li>
    <li><span style='background:#62A7C5;opacity:1;'></span>115 - 132</li>
    <li><span style='background:#348CB1;opacity:1;'></span>133 - 285</li>
    <li><span style='background:#007AA8;opacity:1;'></span>> 285</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

template_nokia_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 60px; top: 0px;'>
<div class='legend-title'>Uses of WIFI hotspots in Paris by device: Nokia</div>
</div>

</body>
</html>

{% endmacro %}"""

In [38]:
district_nokia_map = folium.Map(location=[48.856614, 2.3522219],
                           zoom_start=12, tiles='cartodbpositron')

folium.GeoJson(
    district_nokia_df,
    style_function=lambda x: {
    'fillColor': colormap_nokia(x['properties']['nb_connetion']),
    'color': 'black',
    'weight': 2,
    'fillOpacity': 1
},
    tooltip=folium.GeoJsonTooltip(
        fields=['Code postal', 'nb_connetion'],
        aliases=['Code postal', 'Nb connection'],
        localize=False
    ),
    name='nokia').add_to(district_nokia_map)

for i, v in site_device_df[site_device_df['Constructeur appareil']=='Nokia'].iterrows():
    popup = """
    <h4>%s</h4><br>
    Device : <b>%s</b><br>
    Adresse : <b>%s</b><br>
    Nb connection : <b>%d</b><br>
    """ % (v['Code Site'], v['Constructeur appareil'], v['Adresse'], v['nb_connetion'])
    
    folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=v['nb_connetion'] / 10,
                            tooltip=popup,
                            color='#E9F6FC',
                            fill_color='#E9F6FC',
                            fill=True).add_to(district_nokia_map)

folium.map.CustomPane('labels').add_to(district_nokia_map)
folium.TileLayer('CartoDBPositronOnlyLabels',
                 pane='labels').add_to(district_nokia_map)

macro = MacroElement()
macro._template = Template(template_nokia)
macro2 = MacroElement()
macro2._template = Template(template_nokia_title)
district_nokia_map.get_root().add_child(macro)
district_nokia_map.get_root().add_child(macro2)